# **1. Clone GIT and Prepare Files**

In [ ]:
# Clone DeepCut Repo
!git clone https://github.com/huak95/deepcut.git
# Clone Additional CORPUS
!git clone https://github.com/korakot/corpus.git

Cloning into 'deepcut'...
remote: Enumerating objects: 531, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 531 (delta 16), reused 0 (delta 0), pack-reused 505
Receiving objects: 100% (531/531), 11.54 MiB | 43.61 MiB/s, done.
Resolving deltas: 100% (281/281), done.
Cloning into 'corpus'...
remote: Enumerating objects: 26836, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 26836 (delta 16), reused 23 (delta 9), pack-reused 26806
Receiving objects: 100% (26836/26836), 62.86 MiB | 23.19 MiB/s, done.
Resolving deltas: 100% (670/670), done.
Checking out files: 100% (27996/27996), done.


In [ ]:
# Download and UnZIP Kaggle Word SegmentDataSet (LST20)
!gdown --id '1OxAPWlilK1gAl0_hDlMEHAUe_r-WogoG'
!unzip '/content/dataset.zip'

Streaming output truncated to the last 5000 lines.
  inflating: dataset/train/T10215.txt  
  inflating: __MACOSX/dataset/train/._T10215.txt  
  inflating: dataset/train/T00760.txt  
  inflating: __MACOSX/dataset/train/._T00760.txt  
  inflating: dataset/train/T10573.txt  
  inflating: __MACOSX/dataset/train/._T10573.txt  
  inflating: dataset/train/T00774.txt  
  inflating: __MACOSX/dataset/train/._T00774.txt  
  inflating: dataset/train/T10567.txt  
  inflating: __MACOSX/dataset/train/._T10567.txt  
  inflating: dataset/train/T11679.txt  
  inflating: __MACOSX/dataset/train/._T11679.txt  
  inflating: dataset/train/T12370.txt  
  inflating: __MACOSX/dataset/train/._T12370.txt  
  inflating: dataset/train/T12416.txt  
  inflating: __MACOSX/dataset/train/._T12416.txt  
  inflating: dataset/train/T10201.txt  
  inflating: __MACOSX/dataset/train/._T10201.txt  
  inflating: dataset/train/T12371.txt  
  inflating: __MACOSX/dataset/train/._T12371.txt  
  inflating: dataset/train/T11678.txt  

# **2. Install Dependency**

In [ ]:
# install DeepCut Requirements
!pip install -r /content/deepcut/requirements.txt -qq

     |████████████████████████████████| 462 kB 5.1 MB/s 


In [ ]:
%cd deepcut/deepcut
%ls

/content/deepcut/deepcut
deepcut.py  __init__.py  model.py  stop_words.py  train.py  utils.py  weight/


In [ ]:
%pwd

'/content/deepcut/deepcut'

# **3. Import Library**

In [ ]:
#!/usr/bin/env python
# encoding: utf-8
import os
from glob import glob
import pandas as pd
from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import utils
from train import *

if __package__ != 'deepcut':
    from utils import create_n_gram_df, CHAR_TYPE_FLATTEN, CHARS_MAP, CHAR_TYPES_MAP
    from model import get_convo_nn2
else:
    from .utils import create_n_gram_df, CHAR_TYPE_FLATTEN, CHARS_MAP, CHAR_TYPES_MAP
    from .model import get_convo_nn2


# **4. Load LST20 Corpus**

### DataFrame แบบแบ่ง by word (1 row คือ 1 word)

ผู้เขียนได้เพิ่ม column sentenceBound เข้าไป เพื่อให้ง่ายต่อการใช้ DataFrame (ปกติ sentence จะ bound ด้วย blank line)
                                                                                                                                                                  
***value ใน sentenceBound***

*   *B-SEN* สำหรับ คำที่เริ่มต้นประโยค
*   *I-SEN* สำหรับ คำที่ไม่ใช่คำจบประโยค
*  *E-SEN* สำหรับ คำจบประโยค

## **!!!! _ (วรรค) จะถูกนับเป็น I-SEN**






In [ ]:
#read file by glob
path_train = glob('/content/dataset/train/*')
path_test  = glob('/content/dataset/test/*')
path_eval  = glob('/content/dataset/eval/*')
print(path_train[1])

/content/dataset/train/T11295.txt


In [ ]:
def create_dataframe(list_path):
    data_list = []
    for item in list_path:
        with open(item, 'r') as f:
            for line in f:
                if line == '\n':
                    continue
                line = line.strip().split('\t')
                data_list.append(line)
    df = pd.DataFrame(data_list, columns = ['word', 'pos', 'entity-name', 'sentence-seg'])
    df['pos'] = df['pos'].fillna('O')
    df['entity-name'] = df['entity-name'].fillna('O')
    df['sentence-seg'] = df['sentence-seg'].fillna('O')
    return df

In [ ]:
# train_df = create_dataframe(path_train)
# test_df  = create_dataframe(path_test)
# eval_df  = create_dataframe(path_eval)

In [ ]:
# print(f'train_df.shape: {train_df.shape}')
# train_df.head()

In [ ]:
!mkdir '/content/LST20/'

In [ ]:
def mkdir_not_error(path):
    try:
        os.mkdir(path)
    except:
        pass

def df_to_csv(paths):
    filename = paths[0].split('/')[-2]
    print(f'filename: {filename}')
    train_df = create_dataframe(paths)
    train_array = train_df['word'].to_numpy()
    df_train = create_char_dataframe(train_array)
    mkdir_not_error('/content/LST20/')
    mkdir_not_error(f'/content/LST20/{filename}')
    df_train.to_csv(f'/content/LST20/{filename}/df_best_LST20_{filename}.csv')

#### Create all CSV in DeepCut formats

In [ ]:
!rm -rf '/content/LST20'

df_to_csv(path_train)
df_to_csv(path_test)
df_to_csv(path_eval)

filename: train
filename: test
filename: eval


# **5. FineTune Model**

## Define Function

In [ ]:
import tensorflow

In [ ]:
#!/usr/bin/env python
# encoding: utf-8
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, \
    Concatenate, Flatten, SpatialDropout1D, \
    BatchNormalization, Conv1D, Maximum, ZeroPadding1D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.optimizers import Adam

def conv_unit(inp, n_gram, no_word=200, window=2):
    out = Conv1D(no_word, window, strides=1, padding="valid", activation='relu')(inp)
    out = TimeDistributed(Dense(5, input_shape=(n_gram, no_word)))(out)
    out = ZeroPadding1D(padding=(0, window - 1))(out)
    return out

def get_model_nn(no_word=200, n_gram=21, no_char=178):
    input1 = Input(shape=(n_gram,))
    input2 = Input(shape=(n_gram,))

    a = Embedding(no_char, 32, input_length=n_gram)(input1)
    a = SpatialDropout1D(0.15)(a)
    a = BatchNormalization()(a)

    a_concat = []
    for i in range(1,9):
        a_concat.append(conv_unit(a, n_gram, no_word, window=i))
    for i in range(9,12):
        a_concat.append(conv_unit(a, n_gram, no_word - 50, window=i))
    a_concat.append(conv_unit(a, n_gram, no_word - 100, window=12))
    a_sum = Maximum()(a_concat)

    b = Embedding(12, 12, input_length=n_gram)(input2)
    b = SpatialDropout1D(0.15)(b)

    x = Concatenate(axis=-1)([a, a_sum, b])
    #x = Concatenate(axis=-1)([a_sum, b])
    x = BatchNormalization()(x)

    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[input1, input2], outputs=out)
    opt = Adam(learning_rate=0.001)
    
    model.compile(optimizer=opt,
                  loss='Adam', 
                  metrics=['acc'],
                  )
    return model

In [ ]:
model = get_model_nn()
model.summary()

## Prepare Feature

In [ ]:
# x_char, x_type, y = prepare_feature('/content/deepcut/deepcut/cleaned_data')

In [ ]:
# utils.CHARS_MAP.get(x_char, 80)

In [ ]:
!mkdir /content/LST20/val/
!cp -r /content/LST20/eval/df_best_LST20_eval.csv /content/LST20/val/df_best_LST20_val.csv

In [ ]:
%%time
# weight_path_load = '/content/drive/Shareddrives/SuperAI_Dataset/NLP/training_fucing_longnight/cnn_without_ne_ab_finetune.h5'
weight_path_load = '/content/drive/Shareddrives/SuperAI_Dataset/NLP/training_fucing_longnight/cnn_finetune_1_timestep.h5'

model = train_model('/content/LST20', 
                    verbose=1, 
                    weight_path = weight_path_load, # Change Model Path Duewwww!!!
                    # train_params= [(10, 256), (3, 512), (3, 2048), (3, 4096), (3, 8192)],
                    # train_params= [(3, 2048), (3, 4096), (3, 8192)],
                    train_params= [(3, 8192)],
                    model = get_model_nn()
                    )

train with 3 epochs and 8192 batch size
Epoch 1/3


ValueError: ignored

In [ ]:
?train_model

# Save Model

In [ ]:
!pwd

In [ ]:
model.save_weights('../../last_weight/last_weight')
model.save('../../last_weight/last_model.h5')

In [ ]:
!zip -r ../../content/last_weight.zip ../../content/last_weight

In [ ]:
!mv ../../last_weight.zip /content/drive/Shareddrives/SuperAI_Dataset/NLP/training_fucing_longnight

In [ ]:
!mv ../../last_weight/last_model.h5 /content/drive/Shareddrives/SuperAI_Dataset/NLP/training_fucing_longnight

In [ ]:
!pwd

In [ ]:
!mv ../../content/deepcut/deepcut/weight/cnn_without_ne_ab.h5 /content/drive/Shareddrives/SuperAI_Dataset/NLP/training_fucing_longnight

# Load Model

#### **Load test**

In [ ]:
%cd ../../
!gdown --id '1zcSGEPXk-DotYSgkU3vHxLiy6kmYqNWX'
!unzip 'nlp-1-word-segmentation-without-dataset.zip'
%cd deepcut/deepcut/

/content
Downloading...
From: https://drive.google.com/uc?id=1zcSGEPXk-DotYSgkU3vHxLiy6kmYqNWX
To: /content/nlp-1-word-segmentation-without-dataset.zip
100% 101k/101k [00:00<00:00, 82.5MB/s]
Archive:  nlp-1-word-segmentation-without-dataset.zip
  inflating: nlp-1-word-segmentation/ws_list.txt  
  inflating: nlp-1-word-segmentation/ws_sample_submission.csv  
  inflating: nlp-1-word-segmentation/ws_test.txt  
/content/deepcut/deepcut


In [ ]:
# Load Test.txt File
path = 'nlp-1-word-segmentation/'
f = open(path + 'ws_test.txt', "r")
txtData = (f.read())
f.close()

In [ ]:
txtData

'ที่ยังสถานการณ์ยังไม่คลี่คลายอาจส่งผลกระทบการค้าชายแดนไทย - กัมพูชา ว่า เท่าที่เข้าไปดำเนินการตรวจสอบยังไม่พบว่ามีการปิดด่านบริเวณดังกล่าว และที่สำคัญการค้าระหว่างไทยและกัมพูชาส่วนใหญ่จะมีปริมาณมากที่ด่านปอยเปต อ. อรัญประเทศ จ. สระแก้ว มากกว่าด่านอื่น ๆ ส่วนที่กันทรลักษ์เป็นด่านขนาดเล็กจะเน้นด้านท่องเที่ยว จึงไม่น่าส่งผลกระทบต่อบรรยากาศการค้าชายแดนไทย - กัมพูชาแต่อย่างใด อย่างไรก็ตาม แม้จะมีปัญหาความขัดแย้งเรื่องเขาพระวิหาร แต่ปริมาณการค้าชายแดนไทย - กัมพูชาในปีที่ผ่านมายังคงเพิ่มขึ้น โดยมีจำปริมาณทั้งสิ้น 48,406 ล้านบาท เพิ่มขึ้นเมื่อเทียบกับปี 2549 ร้อยละ 0.2 โดยเป็นการส่งออกจากไทยทั้งสิ้น 46,718 ล้านบาท ขณะที่มีการนำเข้าสินค้าจากกัมพูชาเพียง 1,689 ล้านบาทเท่านั้น ทั้งนี้ ในช่วง 5 เดือนแรกปีนี้ ( ม.ค. - พ.ค. 2551 ) การค้าทั้งสองประเทศมีมูลค่ารวมทั้งสิ้น 29,677 ล้านบาท เพิ่มขึ้นเมื่อเทียบกับช่วงเดียวกันของปีที่ผ่านมาร้อยละ 59.2 โดยไทยส่งออกมูลค่าทั้งสิ้น 28,680 ล้านบาท และมีการนำเข้าเพียง 997 ล้านบาท ถกนอกรอบ " ไทย - เขมร " 8 ชม. สรุป " ตรึงกำลังทหาร - ห้ามยิง " ผลการประชุมร่วมไทย - 

In [ ]:
def tokenize_word(text, weight=weight_path_load):
    pretrained_model = get_convo_nn2()
    pretrained_model.load_weights(weight)
    x_char, x_type = utils.create_feature_array(text)

    y_predict = pretrained_model.predict([x_char, x_type])
    y_predict = (y_predict.ravel() > 0.5).astype(int)
    word_end = y_predict[1:].tolist() + [1]
    tokens = []
    word = ''
    for char, w_e in zip(text, word_end):
        word += char
        if w_e:
            tokens.append(word)
            word = ''
    return tokens

#### Fine Tune

In [ ]:
resDeepcut_custom = tokenize_word(txtData, weight=weight_path_load)

In [ ]:
print(resDeepcut_custom)

['ที่', 'ยัง', 'สถานการณ์', 'ยัง', 'ไม่', 'คลี่คลาย', 'อาจ', 'ส่ง', 'ผล', 'กระทบ', 'การ', 'ค้า', 'ชายแดน', 'ไทย - กัมพูชา ว่า เท่า', 'ที่', 'เข้า', 'ไป', 'ดำเนินการ', 'ตรวจสอบ', 'ยัง', 'ไม่', 'พบ', 'ว่า', 'มีการ', 'ปิด', 'ด่าน', 'บริเวณ', 'ดังกล่าว และ', 'ที่', 'สำคัญ', 'การ', 'ค้า', 'ระหว่าง', 'ไทย', 'และ', 'กัมพูชา', 'ส่วนใหญ่', 'จะ', 'มี', 'ปริมาณ', 'มาก', 'ที่', 'ด่าน', 'ปอยเปต ', 'อ. อรัญประเทศ จ. สระแก้ว ', 'มาก', 'กว่า', 'ด่าน', 'อื่น ', 'ๆ ส่วน', 'ที่', 'กัน', 'ทรลักษ์', 'เป็น', 'ด่าน', 'ขนาด', 'เล็ก', 'จะ', 'เน้น', 'ด้าน', 'ท่องเที่ยว จึง', 'ไม่', 'น่า', 'ส่ง', 'ผล', 'กระทบ', 'ต่อ', 'บรรยากาศ', 'การ', 'ค้า', 'ชายแดน', 'ไทย - กัมพูชา', 'แต่อย่างใด อย่างไรก็ตาม ', 'แม้', 'จะ', 'มี', 'ปัญหา', 'ความ', 'ขัดแย้ง', 'เรื่อง', 'เขา', 'พระวิหาร ', 'แต่', 'ปริมาณ', 'การ', 'ค้า', 'ชายแดน', 'ไทย - กัมพูชา', 'ใน', 'ปี', 'ที่', 'ผ่าน', 'มา', 'ยังคง', 'เพิ่ม', 'ขึ้น โดย', 'มี', 'จำ', 'ปริมาณ', 'ทั้งสิ้น 48,406 ', 'ล้าน', 'บาท ', 'เพิ่ม', 'ขึ้น', 'เมื่อ', 'เทียบ', 'กับ', 'ปี 2549', ' ร้อย', 'ล

In [ ]:
weight_path_load

'/content/drive/Shareddrives/SuperAI_Dataset/NLP/training_fucing_longnight/cnn_without_ne_ab_finetune.h5'

#### Original

In [ ]:
resDeepcut_custom2 = tokenize_word(txtData, weight='/content/drive/Shareddrives/SuperAI_Dataset/NLP/training_fucing_longnight/cnn_without_ne_ab_original.h5')
print(resDeepcut_custom2)

['ที่', 'ยัง', 'สถานการณ์', 'ยัง', 'ไม่', 'คลี่คลาย', 'อาจ', 'ส่ง', 'ผล', 'กระทบ', 'การ', 'ค้า', 'ชายแดน', 'ไทย', ' ', '-', ' ', 'กัมพูชา', ' ', 'ว่า', ' ', 'เท่า', 'ที่', 'เข้า', 'ไป', 'ดำเนิน', 'การ', 'ตรวจสอบ', 'ยัง', 'ไม่', 'พบ', 'ว่า', 'มี', 'การ', 'ปิด', 'ด่าน', 'บริเวณ', 'ดัง', 'กล่าว', ' ', 'และ', 'ที่', 'สำคัญ', 'การ', 'ค้า', 'ระหว่าง', 'ไทย', 'และ', 'กัมพูชา', 'ส่วน', 'ใหญ่', 'จะ', 'มี', 'ปริมาณ', 'มาก', 'ที่', 'ด่านปอยเปต', ' ', 'อ. อรัญประเทศ', ' ', 'จ. สระแก้ว', ' ', 'มาก', 'กว่า', 'ด่าน', 'อื่น', ' ', 'ๆ', ' ', 'ส่วน', 'ที่', 'กัน', 'ทรลักษ์', 'เป็น', 'ด่าน', 'ขนาด', 'เล็ก', 'จะ', 'เน้น', 'ด้าน', 'ท่องเที่ยว', ' ', 'จึง', 'ไม่', 'น่า', 'ส่ง', 'ผล', 'กระทบ', 'ต่อ', 'บรรยากาศ', 'การ', 'ค้า', 'ชายแดน', 'ไทย', ' ', '-', ' ', 'กัมพูชา', 'แต่อย่างใด', ' ', 'อย่าง', 'ไร', 'ก็ตาม', ' ', 'แม้', 'จะ', 'มี', 'ปัญหา', 'ความ', 'ขัดแย้ง', 'เรื่อง', 'เขาพระวิหาร', ' ', 'แต่', 'ปริมาณ', 'การ', 'ค้า', 'ชายแดน', 'ไทย', ' ', '-', ' ', 'กัมพูชา', 'ใน', 'ปี', 'ที่', 'ผ่าน', 'มา', 'ยังคง', 'เพ

In [ ]:
res_LST20_oskut = tokenize_word(txtData, weight='/content/drive/Shareddrives/SuperAI_Dataset/NLP/training_fucing_longnight/lst20_rmsprop_LSTM_Attension_224_32_weight.h5')
print(res_LST20_oskut)

ValueError: ignored

In [ ]:
resDeepcut_custom = tokenize_word(txtData, weight=weight_path_load)

In [ ]:
df_DeepCut = pd.DataFrame(resDeepcut_custom, columns=['word']) 
df_DeepCut

,word
0,ที่
1,ยัง
2,สถานการณ์
3,ยัง
4,ไม่
...,...
7359,เพื่อนบ้าน
7360,เป็น
7361,ที่
7362,เรียบร้อย


In [ ]:
def split(word):
    return list(word)

def text_to_submission(test_text):
    data = []
    label = []
    id = []
    for i in test_text:
        a = split(i)
        if i == ' ':
            label.append('O')
            continue

        for index, cha_inside in enumerate(a):
            if index == 0:
                label.append('B_WORD')
            elif index == len(a)-1:
                label.append('E_WORD')
            elif cha_inside == ' ':
                label.append('O')
            else:
                label.append('I_WORD')

        data.extend(a)

    label = pd.DataFrame(label)
    data = pd.DataFrame(data)
    data_submission = pd.concat([data, label], axis=1)
    data_submission.columns = ['Id', 'Predicted']

    id1 = data_submission['Id']
    for i, n in enumerate(id1):
        id.append(i+1)
    id = pd.DataFrame(id)

    final_submission = pd.concat([id, label], axis=1)
    final_submission.columns = ['Id', 'Predicted']

    return final_submission


In [ ]:
def dropO(df):
    str1 = df.columns[1]
    return df[df[str1] != 'O']

In [ ]:
df_submition = dropO(text_to_submission(resDeepcut_custom))
df_submition

,Id,Predicted
0,1,B_WORD
1,2,I_WORD
2,3,E_WORD
3,4,B_WORD
4,5,I_WORD
...,...,...
37243,37244,E_WORD
37244,37245,B_WORD
37245,37246,I_WORD
37246,37247,I_WORD


In [ ]:
df_submition.to_csv('submission_harrycut7_100%.csv', index=False)